In [19]:
import pandas as pd
from pathlib import Path
from datetime import datetime

RAW_DATA = Path("../data/raw/ethiopia_fi_unified_data.xlsx")
CODES = Path("../data/raw/reference_codes.xlsx")

# Load sheets
data = pd.read_excel(RAW_DATA, sheet_name="ethiopia_fi_unified_data")
impact_links = pd.read_excel(RAW_DATA, sheet_name="Impact_sheet")
codes = pd.read_excel(CODES)

print(data.shape, impact_links.shape)


(43, 34) (14, 35)


In [20]:
for col in ["observation_date", "event_date", "collection_date"]:
    if col in data.columns:
        data[col] = pd.to_datetime(data[col], errors="coerce")


/var/folders/nf/drxnv59536v7bnknmx3c_1zw0000gn/T/ipykernel_9907/3707553364.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data[col] = pd.to_datetime(data[col], errors="coerce")


In [22]:
data["record_type"].value_counts()
pd.crosstab(data["record_type"], data["pillar"])
impact_links.head()

,record_id,parent_id,record_type,category,pillar,indicator,indicator_code,indicator_direction,value_numeric,value_text,value_type,unit,observation_date,period_start,period_end,fiscal_year,gender,location,region,source_name,source_type,source_url,confidence,related_indicator,relationship_type,impact_direction,impact_magnitude,impact_estimate,lag_months,evidence_basis,comparable_country,collected_by,collection_date,original_text,notes
0,IMP_0001,EVT_0001,impact_link,NaN,ACCESS,Telebirr effect on Account Ownership,NaN,NaN,15.0,NaN,percentage,%,2021-05-17,NaN,NaN,NaN,all,national,NaN,NaN,NaN,NaN,medium,ACC_OWNERSHIP,direct,increase,high,15.0,12,literature,Kenya,Example_Trainee,2025-01-20,NaN,Kenya M-Pesa showed +20pp over 5 years
1,IMP_0002,EVT_0001,impact_link,NaN,USAGE,Telebirr effect on Telebirr Users,NaN,NaN,NaN,NaN,count,users,2021-05-17,NaN,NaN,NaN,all,national,NaN,NaN,NaN,NaN,high,USG_TELEBIRR_USERS,direct,increase,high,NaN,3,empirical,NaN,Example_Trainee,2025-01-20,NaN,Direct subscriber acquisition
2,IMP_0003,EVT_0001,impact_link,NaN,USAGE,Telebirr effect on P2P Transactions,NaN,NaN,25.0,NaN,percentage,%,2021-05-17,NaN,NaN,NaN,all,national,NaN,NaN,NaN,NaN,medium,USG_P2P_COUNT,direct,increase,high,25.0,6,empirical,NaN,Example_Trainee,2025-01-20,NaN,New digital payment channel
3,IMP_0004,EVT_0002,impact_link,NaN,ACCESS,Safaricom effect on 4G Coverage,NaN,NaN,15.0,NaN,percentage,%,2022-08-01,NaN,NaN,NaN,all,national,NaN,NaN,NaN,NaN,medium,ACC_4G_COV,direct,increase,medium,15.0,12,empirical,NaN,Example_Trainee,2025-01-20,NaN,Network investment from competition
4,IMP_0005,EVT_0002,impact_link,NaN,AFFORDABILITY,Safaricom effect on Data Affordability,NaN,NaN,-20.0,NaN,percentage,%,2022-08-01,NaN,NaN,NaN,all,national,NaN,NaN,NaN,NaN,medium,AFF_DATA_INCOME,indirect,decrease,medium,-20.0,12,literature,Rwanda,Example_Trainee,2025-01-20,NaN,Competition typically reduces prices


In [23]:
impact_links["record_type"] = "impact_link"


In [24]:
unified = pd.concat([data, impact_links], ignore_index=True)


/var/folders/nf/drxnv59536v7bnknmx3c_1zw0000gn/T/ipykernel_9907/765670567.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  unified = pd.concat([data, impact_links], ignore_index=True)


In [25]:
new_obs = pd.DataFrame([{
    "record_type": "observation",
    "pillar": "usage",
    "indicator": "Active mobile money accounts (% of adults)",
    "indicator_code": "mm_active_pct",
    "value_numeric": 18.2,
    "observation_date": pd.Timestamp("2023-12-31"),
    "source_name": "GSMA Mobile Money Metrics",
    "source_url": "https://www.gsma.com/mobilemoney/",
    "confidence": "medium",
    "original_text": "18.2% of adults actively used mobile money in Ethiopia in 2023",
    "collected_by": "Zemicahel Abraham",
    "collection_date": pd.Timestamp.today(),
    "notes": "Explains ownership vs usage gap"
}])

unified = pd.concat([unified, new_obs], ignore_index=True)


In [27]:
OUTPUT = Path("../data/processed/ethiopia_fi_unified_data_enriched.xlsx")
OUTPUT.parent.mkdir(parents=True, exist_ok=True)

with pd.ExcelWriter(OUTPUT, engine="xlsxwriter") as writer:
    unified[unified["record_type"] != "impact_link"] \
        .to_excel(writer, sheet_name="ethiopia_fi_unified_data", index=False)

    unified[unified["record_type"] == "impact_link"] \
        .to_excel(writer, sheet_name="Impact_sheet", index=False)

print("✅ Enriched dataset saved")


✅ Enriched dataset saved
